In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [1]:
import numpy as np
import seaborn as sns
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import SGDClassifier, LinearRegression
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix, precision_recall_fscore_support, mean_squared_error, mean_absolute_error

In [1]:
df_data = pd.read_csv('../input/kickstarter-projects/ks-projects-201801.csv')
df_data.head(30)

In [1]:
df_data.describe()

ID: input, -  
name: input, -  
category:input, 質的変数  
main_category: input, 質的変数  
currency: input, 質的変数  
deadline: input, 量的変数？  
goal: input, 量的変数 -> usd_goal_real  
launched: input, 量的変数？  
pledged: target, 量的変数 -> usd_pledged_real  
state: target, 質的変数  
backers: target, 量的変数  
country: input, 質的変数  
usd_pledged: target, 量的変数 -> usd_pledged_real  
usd_pledged_real: target, 量的変数  
usd_goal_real: input, 量的変数

In [1]:
#convert qualitative variable to dummy variable
df_data = pd.get_dummies(df_data, columns = ["category", "main_category", "currency", "country", "state"])
df_data["pledged_per_goal"] = df_data["usd_pledged_real"] / df_data["usd_goal_real"]
df_data.head()

In [1]:
sns.heatmap(pd.concat([
    df_data.loc[:, df_data.columns.str.startswith('category')],
    df_data.loc[:, df_data.columns.str.startswith('state')],
    df_data["usd_pledged_real"],
    df_data["pledged_per_goal"]
], axis = 1).corr())

pd.concat([
    df_data.loc[:, df_data.columns.str.startswith('category')],
    df_data.loc[:, df_data.columns.str.startswith('state')],
    df_data["usd_pledged_real"],
    df_data["pledged_per_goal"]
], axis = 1).corr()

In [1]:
sns.heatmap(pd.concat([
    df_data.loc[:, df_data.columns.str.startswith('main_category')],
    df_data.loc[:, df_data.columns.str.startswith('state')],
    df_data["usd_pledged_real"],
    df_data["pledged_per_goal"]
], axis = 1).corr())

pd.concat([
    df_data.loc[:, df_data.columns.str.startswith('main_category')],
    df_data.loc[:, df_data.columns.str.startswith('state')],
    df_data["usd_pledged_real"],
    df_data["pledged_per_goal"]
], axis = 1).corr()

In [1]:
sns.heatmap(pd.concat([
    df_data.loc[:, df_data.columns.str.startswith('currency')],
    df_data.loc[:, df_data.columns.str.startswith('country')],
    df_data.loc[:, df_data.columns.str.startswith('state')],
    df_data["usd_pledged_real"]
], axis = 1).corr())

pd.concat([
    df_data.loc[:, df_data.columns.str.startswith('currency')],
    df_data.loc[:, df_data.columns.str.startswith('country')],
    df_data.loc[:, df_data.columns.str.startswith('state')],
    df_data["usd_pledged_real"]
], axis = 1).corr()

In [1]:
sns.heatmap(pd.concat([
    df_data.loc[:, df_data.columns.str.startswith('usd_goal_real')],
    df_data.loc[:, df_data.columns.str.startswith('state')],
    df_data["usd_pledged_real"],
    df_data["pledged_per_goal"]
], axis = 1).corr())

pd.concat([
    df_data.loc[:, df_data.columns.str.startswith('usd_goal_real')],
    df_data.loc[:, df_data.columns.str.startswith('state')],
    df_data["usd_pledged_real"],
    df_data["pledged_per_goal"]
], axis = 1).corr()

In [1]:
sns.heatmap(pd.concat([
    pd.to_datetime(df_data["launched"]).map(pd.Timestamp.timestamp),
    pd.to_datetime(df_data["deadline"]).map(pd.Timestamp.timestamp),
    pd.to_datetime(df_data["deadline"]).map(pd.Timestamp.timestamp) - pd.to_datetime(df_data["launched"]).map(pd.Timestamp.timestamp),
    df_data.loc[:, df_data.columns.str.startswith('state')],
    df_data["usd_pledged_real"],
    df_data["pledged_per_goal"]
], axis = 1).corr())

pd.concat([
    pd.to_datetime(df_data["launched"]).map(pd.Timestamp.timestamp),
    pd.to_datetime(df_data["deadline"]).map(pd.Timestamp.timestamp),
    pd.to_datetime(df_data["deadline"]).map(pd.Timestamp.timestamp) - pd.to_datetime(df_data["launched"]).map(pd.Timestamp.timestamp),
    df_data.loc[:, df_data.columns.str.startswith('state')],
    df_data["usd_pledged_real"],
    df_data["pledged_per_goal"]
], axis = 1).corr()

相関を見ただけでは説明力のある説明変数は見つからず。

In [1]:
df_input = pd.concat([
    df_data.loc[:, df_data.columns.str.startswith('category_')],
    df_data.loc[:, df_data.columns.str.startswith('main_category_')],
    df_data.loc[:, df_data.columns.str.startswith('country_')],
    df_data.loc[:, df_data.columns.str.startswith('currency_')],
    df_data.loc[:, df_data.columns.str.startswith('usd_goal_real_')],
    pd.to_datetime(df_data["launched"]).map(pd.Timestamp.timestamp),
    pd.to_datetime(df_data["deadline"]).map(pd.Timestamp.timestamp),
    pd.to_datetime(df_data["deadline"]).map(pd.Timestamp.timestamp) - pd.to_datetime(df_data["launched"]).map(pd.Timestamp.timestamp)
], axis = 1)
df_input.head()

In [1]:
df_target = pd.concat([
    df_data.loc[:, df_data.columns.str.startswith('state_')],
    df_data[["usd_pledged_real", "pledged_per_goal", "backers"]],
], axis = 1)
df_target.head(30)

In [1]:
df_target.describe()

stateの分類

使うtargetのみを抽出

In [1]:
df_target1 = df_data.loc[:, df_data.columns.str.startswith('state_')]

In [1]:
na_data_x = df_input.values
na_data_y1 = np.argmax(df_target1.values, axis = 1)
print(na_data_y1)

In [1]:
X_train, X_test, Y_train, Y_test = train_test_split(na_data_x, na_data_y1, test_size = 0.1)

In [1]:
clf = SGDClassifier(loss = 'log', penalty = 'none', max_iter = 10000, fit_intercept = True, random_state = 1234, tol = 1e-3)
clf.fit(X_train, Y_train)

In [1]:
y_pred = clf.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(Y_test, y_pred)

In [1]:
print('Accuracy: {:.3f}%'.format(100 * accuracy))
print('Precision: {:.3f}%'.format(100 * precision[0]))
print('Recall: {:.3f}%'.format(100 * recall[0]))
print('F1 Score: {:.3f}%'.format(100 * f1_score[0]))

In [1]:
conf_mat = confusion_matrix(Y_test, y_pred)
conf_mat = pd.DataFrame(conf_mat,
                           index = ['正解: canceled', '正解: failed', '正解: live', '正解: successful', '正解: suspended', '正解: undefined'],
                           columns = ['予測: canceled', '予測: failed', '予測: live', '予測: successful', '予測: suspended', '予測: undefined']
)
conf_mat

pledgeの回帰

使うtargetのみを抽出

In [1]:
df_target2 = df_target["usd_pledged_real"]

In [1]:
na_data_x = df_input.values
na_data_y2 = df_target2.values

In [1]:
X_train, X_test, Y_train, Y_test = train_test_split(na_data_x, na_data_y2, test_size = 0.1)

In [1]:
regr = LinearRegression(fit_intercept = True)
regr.fit(X_train, Y_train)

In [1]:
y_pred = regr.predict(X_test)

mse = mean_squared_error(Y_test, y_pred)
mae = mean_absolute_error(Y_test, y_pred)
rmse = np.sqrt(mse)

In [1]:
print("MSE: %s"%round(mse, 3))
print("MAE: %s"%round(mae, 3))
print("RMSE: %s"%round(rmse, 3))

pledgeの回帰(2)

使うtargetのみを抽出

In [1]:
df_target3 = df_target["pledged_per_goal"]

In [1]:
na_data_x = df_input.values
na_data_y3 = df_target3.values

In [1]:
X_train, X_test, Y_train, Y_test = train_test_split(na_data_x, na_data_y3, test_size = 0.1)

In [1]:
regr = LinearRegression(fit_intercept = True)
regr.fit(X_train, Y_train)

In [1]:
df_input.head(10)

In [1]:
y_pred = regr.predict(X_test)

mse = mean_squared_error(Y_test, y_pred)
mae = mean_absolute_error(Y_test, y_pred)
rmse = np.sqrt(mse)

pledged/goal の評価値

In [1]:
print("MSE: %s"%round(mse, 3))
print("MAE: %s"%round(mae, 3))
print("RMSE: %s"%round(rmse, 3))

In [1]:
print("MAE of real value: %s"%round(mean_absolute_error(Y_test, y_pred * X_test[:, -4]), 3))

悪くなった…。